# Binary Search Trees continue

### Searching for an element in the tree

Now that we have our BST structure, and a possibility to print out its content to see that it is behaving as intended. Let us now look at how to search the tree for a specific element based on its key. To do this, we again use the recursive nature of the tree, as well as it's search property: i.e., we only have to check the left subtree *or* the right subtree, but never both.

```C++
public:
    bool contains(int x) {
        /* Check if the whole tree contains a given value */
        return contains(root, x);
    }

private:
    bool contains(Node* subroot, int x) {
    	/* Check if a given subtree contains a given value */

    	// An empty subtree contains nothing       
        if (subroot == nullptr) {
            return false;
        } 

        if (subroot->key == x) {
        	// Match found
        	return true;

        } else if (x < subroot-> key) {
        	// If match exists, it must be in left subtree
        	return contains(subroot->left, x);

        } else {
        	// If match exists, it must be in right subtree
        	return contains(subroot->right, x);
        }
    }
```

Testing our implementation
```C++
BinarySearchTree tree({7, 4, 1, 3, 5, 6, 2, 9, 0, 8});   
cout << tree.contains(5) << endl;
cout << tree.contains(20) << endl;
```
This prints out
```
1
0
```
Here, 1 means true, and 0 means false. This is how `cout` prints out boolean values. If we want it to actually print out true/false, we can use the `<iomanip>` header to impact this by setting the booleanalpha flag:
```C++
BinarySearchTree tree({7, 4, 1, 3, 5, 6, 2, 9, 0, 8});   
cout << boolalpha;
cout << tree.contains(5) << endl;
cout << tree.contains(20) << endl;
```
```
true
false
```


#### Why we call it the "key"

We are now ready to explain why we call it the *key* of the node. A node can potentially contain any data, including a large amount of different data. A node could for example represent a person in a contact list, and contain various data fields such as the phone number, the email, and so forth. For a linked list, the contents of the node doesn't impact where it goes, so we just called it the "data" of the node. 

However, for our binary search tree, we need access to a *key* on which we check if it is smaller or larger than a given value. So the key must be a simple data type that is sortable, like an integer or a string (which can be sorted alphabetically). The node can still contain all the other data as well, but the *key* itself is the piece of information we use to structure our tree. 

For a contact list, the *key* would typically be the name. Thus every contact needs a name, which identifies them and sorts them, while all other pieces of information would be optional and just stored additionally, it wouldn't impact the binary search tree. In a larger data base, you might want to sort people by some sort of identity number instead, for example an employee number. Each node would contain this code as its key, but all the other information would also be stored in the node, it just wouldn't impact the placement within the tree.

After having programmed Python for a while, you might naturally associate "keys" with dictionaries, which store items in key: value pairs. This is the exact same logic we are using now, in fact, binary search trees are a great underlying structure for implementing a dictionary class (altough, not the one Python uses, which is a different structure called a hash table). 

While implementing a dictionary class from the bottom-up on  top of a binary search tree is a great exercise. We won't take the time to do it here. However, it could be a great exercise to learn more about data structures to do so yourself, if you have the time.

### Removing elements

We now turn to removing elements from our BST based on key. If we want to remove elements with a given key, we first have to find the right node, and then somehow remove it without decoupling the entire tree, which can potentially be a bit tricky. In the process of trying to keep the tree in tact, we also have to be certain to keep the search property of the tree in tact as well.

Once we have found the node to be removed, there are three possible cases: the node to be deleted can have (1) no children, i.e., it is a leaf node, (2) it can have one child, or (3) it chan have two children. Let us discuss these three cases seperately:

**(1)** If the node has no children, deleting it is fairly trivial, just set the pointer of its parent to `nullptr` and you are done. This is the same as deleting the last element in a linked list.

**(2)** If the node we are deleting has a single child, things are still fairly easy. We simply need to take the parent's point and point it at its "grandchild". This is like deleting an element from the middle of a linked list.

**(3)** If there are two children, things get a bit more tricky. Imagine for example trying to remove the root of the whole tree, effectively we are cleaving the tree into two completely separated new trees—but we don't want this, we want to keep the remaining noded as a single tree. The challenge is then to find out what should become the *new root* of the tree.

The way we handle this is to find the *inorder successor* of the node to be deleted, i.e., the element that would naturally follow it in a sorted list. The successor will always be found by first stepping a single time to the right (from the node to be deleted), and then stepping continiously to the left untill you reach a node with no left child. We then move the successor up to become the new root by first deleting it from its current location in the tree, and then replacing the node to be actually be deleted by it. As the inorder successor node has no left child (by definition), it has at most one child, and so deleting it will be fairly easy. 

This process is a lot easier to envision with an example, so let us draw a figure:
<img src="fig/bst_removal.png" width=700>
<center><b>Figure 5:</b> Removing a node from a BST can potentially be tricky. In the given example we want to remove the node with value `4`. However, this node has two children, and so we have to find a replacement for it, otherwise the tree would split into two. We choose the inorder succsessor, which in this case is the node with `5`. When we move this node up into the deleted nodes place, we have to be careful not to lose its child, `6` in the process!</center>
    
In the given example, we wanted to remove the root of the whole tree. What happens if we want to remove a different node? Well, any node is the root of it's own subtree, and so you could simply perform the same process on that subtree.

#### Implementing remove

Before we implement the remove itself, we define a `find_min` function to help us find the in-order successor:
```C++
Node* find_min(Node* subroot) {
    /* Find the smallest node of a given subtree */
    Node* minimum = subroot;
    while (minimum->left != nullptr) {
        minimum = minimum->left;
    }
    return minimum;
}
```
Now we turn to the actual removal method, which we again do recursively
***
```C++
public:
    void remove(int x) {
        /* Remove node with key x from whole tree if present 
        
        If the key is not present, we throw an error.
        */
        root = remove(root, x);
        count--;
    }
```
***
```C++
private:
    Node* remove(Node* subroot, int x) {
        /* Remove node with key x from subtree if present */
        if (subroot == nullptr) {
            // Element is not contained in tree, cannot remove
            throw runtime_error("element not in tree");
        }
        
        if (x < subroot-> key) {
            // Keep searching left subtree
            subroot->left = remove(subroot->left, x);
        } else if (x > subroot-> key) {
            // Keep searching right subtree
            subroot->right = remove(subroot->right, x);
        } else {
            // We have found the node to be deleted
            
            if (subroot->left == nullptr and subroot->right == nullptr) {
                // 0 children
                delete subroot;
                return nullptr;
            } else if (subroot->left == nullptr) {
                // 1 child
                Node* child = subroot->right;
                delete subroot;
                return child;
            } else if (subroot->right == nullptr) {
                // 1 child
                Node* child = subroot->left;
                delete subroot;
                return child;
            } else {
                // 2 children
                Node* succsessor = find_min(subroot->right);
                subroot->key = successor->key;
                subroot->right = remove(subroot->right, successor->key);
            }
        }
    }
```

This implementation quite a beast with several nested tests, and a recursive definition! But, when going through and implementing it yourself, just be thourough, and do it incremenetally, step by step, and it all works out in the end.

Note also that we in this example use a slight hack on the 2 children solution. Instead of deleting the actual node, and then moving the successor up, we simply copy the succsessor's key, and then delete the succsessor. This works well if we only store the key. If we stored other information than the key, we would also need to copy that over. Copying these values over instead of moving the nodes around is not strictly speaking optimal, but it's a far easier way to implement it.

#### Testing Remove

Because there are the three different cases: removing a node with 0, 1, or 2 children, we should check all of them. We can use the same tree:
```C++
BinarySearchTree tree({7, 4, 1, 3, 5, 6, 2, 9, 0, 8});   
```

We start by checking a node with no children. This could for example be the biggest node in the list: 9. We write the following thest snippet:
```C++
cout << boolalpha;
cout << "Before removal:" << endl;
cout << "===============" << endl;
cout << ".length(): " << tree.length() << endl;
cout << ".contains(9): " << tree.contains(9) << endl;
cout << ".print():" << endl;
tree.print();

tree.remove(9);

cout << endl << "After removal:" << endl;
cout << "===============" << endl;
cout << ".length(): " << tree.length() << endl;
cout << ".contains(9): " << tree.contains(9) << endl;
cout << ".print():" << endl;
tree.print();
```


Which produces the output (with the print changed to be on a single line:
```
Before removal:
===============
.length(): 10
.contains(9): true
.print(): 0 1 2 3 4 5 6 7 8 9

After removal:
===============
.length(): 9
.contains(9): false
.print(): 0 1 2 3 4 5 6 7 8 
```

To check removal of a node with a single child, we simply change the example of 9, to for example 3, which has a single child. Try it yourself. 

Lastly, we must check the case of a node with two elements, here we can choose the root itself for example, which would be 7. Or a different node with two children, for example 4. Try it yourself, and verify that my, or your own, implementation is valid.

## Cost of operations

We have now gone through and implemented a binary search tree class, and defined the most important operations, namely: `insert`, `remove`, `contains` and inorder traversal in the form of `print`. Let us now analyze the performance of our data structure. Let us start with  the `contains` method, which is a simple search, as this was the original motivation for the search tree.

So, how many operations does our `contains` use? First, let us look back to our implementation:

***
```C++
public:
    bool contains(int x) {
        /* Check if the whole tree contains a given value */
        return contains(root, x);
    }

private:
    bool contains(Node* subroot, int x) {
    	/* Check if a given subtree contains a given value */

    	// An empty subtree contains nothing       
        if (subroot == nullptr) {
            return false;
        } 

        if (subroot->key == x) {
        	// Match found
        	return true;

        } else if (x < subroot-> key) {
        	// If match exists, it must be in left subtree
        	return contains(subroot->left, x);

        } else {
        	// If match exists, it must be in right subtree
        	return contains(subroot->right, x);
        }
    }
```
***

While this method might look very complicated, note that as we search for an element in the tree, we only ever move down from the root, never back up. For each step, we choose either the left or right branch, and stick with our decision. This works because of the tree's search property. If there is no match in our tree, then we need to keep looking untill we hit a leaf node in our tree. But how many operations is that? Well, that depends on the *height* of the tree. To understand the concept of height, we also need to discuss *balance*.


#### Tree height and tree balance

Earlier in the lecture, we explained that a tree does not remember its insertion order, but this doesn't mean that the insertion order is unimportant. Depending on the order elements are inserted in, a binary search tree can have a very different structure. We say that a tree is "balanced" if it's node spread out evenly across all branches. While if the nodes start clumping up towards one side of the tree, we say the tree becomes unbalanced.

As an example, let us make two trees, both consisting of the numbers 1 through 7. But we build our trees by inserting the elements in different orders:

1. Insert in order: 4, 6, 5, 2, 1, 7, 3
2. Insert in order: 1, 2, 3, 4, 5, 6, 7

If you go through with pen and paper and construct these two trees out, you will find that tree (1) becomes perfectly balanced, i.e., it fills up all layers perfectly. While (2) becomes "perfectly" unbalanced, with all nodes using the right branch. Because the left branch is never used, our structure doesn't branch at all, and we have effectively just made a linked list! I say "perfectly" in air quotes, because we *want* our tree to be balanced, so there is nothing perfect about this unbalanced tree.

<img src="fig/bst_balance.png" width=600>
<center><b>Figure 6:</b> Two trees consisting of the same elements, but with a different structure. On the left we see a perfectly balanced tree, meaning all branches are filled out at every level. On the right we have a very unbalaced tree, with only the right branches being used, and none of the left branches.</center>

Now, what is the *height* of these two trees? First we need to define the height of a tree:
> The height of a tree structure is the longest path from the top to bottom (root to a leaf) going only down.

So counting, we see that the height of the balanced tree is 3, because there are three "layers", and it takes three steps to go from the root to the bottom. For the unbalanced tree however, the height is 7, as we have to step through all the nodes to reach the bottom of the tree.

Now, let us extend this thinking to a larger, tree with $n$ nodes, what will the height of such a tree be if it is either balanced or unbalanced? The balanced tree will have a height on the order of $\log_2(n)$, because for each new layer we add to the tree, we can fit twice as many nodes, so because the structure is branching, the height grows less and less as the number of nodes increases. You will notice this quickly, try drawing a large balanced tree on a sheet of paper and you will run out of width long before you fill the height of the sheet. For an unbalanced tree however, the height will be on the order of $n$, because each new node you add to the tree adds to the height.

Of course, most real binary search trees won't be perfectly balanced or unbalanced, but lie somewhere in between. Still, for most realistic use cases, the tree will actually turn out be quite well balanced, meaning its height grows as $\mathcal{O}(\log n)$ (Big Oh doesn't care about the base of the logarithm), though we should be careful using BSTs in situations where they might tend to become unbalanced due to the nature of a problem. So if we can assume that the input come in more or less random order, then the tree will be balanced.

#### Cost of searching a BST

So back to the question at hand. We asked: 
* How many operations to check whether our tree contains a given value?

As our search algorithm only moves downward, and never back up in the tree, the *worst case* for our algorithm will be the height of the tree. If there is no match in the tree, the answer is still the height of the tree, as we only need to reach the bottom of the tree to know there is no match. So for a balanced tree, the worst case runtime of searching is $\mathcal{O}(\log n)$, much better than the $\mathcal{O}(n)$ of arrays and linked lists. The average case is also $\mathcal{O}(\log n)$ (why?).

Now, we assumed that our tree was balanced, but this might not always be the case, so for a general BST we say that the worst case is that our tree is "perfectly" unbalanced, in which case it acts like a linked list with a cost of searching of $\mathcal{O}(n)$, but on average, a general tree will tend to be fairly balanced, so the average cose of a general BST is $\mathcal{O}(\log n)$.

#### Cost of traversing the tree

What about the cost of traversing the the elements of the tree, for example to print them out. While the algorithm to do so was a bit trickier than the ones for the linked list or the array, the fact is still that we visit each node of the tree exactly once when traversing it, so the cost of traversing a tree is $\mathcal{O}(n)$, same as for the array and linked list structures.

#### Cost of inserting and deleting

What about adding a new node to the tree? When we want to add a new node, we first have to find where it goes, which is effectively a *search* operation (why?), so first we have the cost of the search, and then adding the new node is $\mathcal{O}(1)$. So for a balanced tree the cost of an insert is $\mathcal{O(\log n)}$.

Deleting a node is very similar, first we need to find the node to be deleted, and then we carry out a finite number of steps, meaning the removal itself is $\mathcal{O}(1)$, but the search is $\mathcal{O}(\log n)$, so removal is also $\mathcal{O}(\log n)$ in total. For an unbalanced tree however, the search becomes linear, so in those cases, we actually get a cost of $\mathcal{n}$.

| Algorithm | Average Case          | Worst case       |
| --------- | --------------------- | ---------------- |
| Search	| $\mathcal{O}(\log n)$ | $\mathcal{O}(n)$ |
| Insert	| $\mathcal{O}(\log n)$ | $\mathcal{O}(n)$ |
| Delete	| $\mathcal{O}(\log n)$ | $\mathcal{O}(n)$ |
| Traverse  | $\mathcal{O}(n)$      | $\mathcal{O}(n)$ |

Now, looking at these costs, we see that there is a huge difference between a balanced tree, which generally has a scaling of $\mathcal{O}(\log n)$ and an unbalanced tree, which has a scaling of $\mathcal{O}(n)$. This is a big difference because a logarithmic growth is *much* slower than a linear growth. 

Put another way, for binary search trees to perform optimally, they have to be balanced. We will return to this point later, to show how we can ensure trees are balanced.

## Binary Search Tree Sort

Now that we have implemented a BST, and analyzed its behavior, let us see a use case of a binary search tree—sorting data.

Recall that when we printed the contents of our tree, it came out in sorted order, a natural consequence of the inorder traveersal we did. So if we have an unsorted list of numbers, we can effectively sort it by loading them into a BST, and reading them out again! This might sound like a horribly inefficient way to do things, but it turns out its actually quite efficient!

Because our sorting is based on the binary search tree structure, we can refer to it as a binary search tree sort, or a BST sort for short.

#### Implementing BST sort
The first thing we want to to is add a method to our `BinarySearchTree` class to get the numbers stored in it, in an in-order output other than printing. We could for example add a method to read out the data stored in it as a vector:
```C++
public:
    vector<int> get_vector() {
        /*Return all elements of tree in order as vector */
        vector<int> numbers;
        add_values(root, numbers);
        return numbers;
    }

private:
    void add_values(Node* subroot, vector<int>& numbers) {
        // Add all keys in subtree to given vector
        if (subroot == nullptr) {
            return;
        }
        
        add_values(subroot->left, numbers);
        numbers.push_back(subroot->key);
        add_values(subroot->right, numbers);
    }
```
This algorithm is exactly the same as our print method implemented earlier. We do in-order traversal of the tree. The only difference is that now we add the data of each node to the vector, instead of printing it out.

With our `get_vector` implemented, we can now create a stand-alone function that sorts a list of numbers
***
```C++
vector<int> bst_sort(vector<int> input) {
    BinarySearchTree bst(input);
    return bst.get_vector();
}
```
***

#### Cost of BST Sort

What is the cost of our algorithm? Say we want to sort a list of $n$ numbers. To sort them, we do two steps:
1. Insert the $n$ elements into the tree
2. Traverse the tree to append the sorted numbers to a vector

The first step is $n$ insertions into the BST, and in the average case, a single insertion costs $\mathcal{O}(\log n)$. The total cost of doing all the $n$ insertions is thus
$$n \cdot \mathcal{O}(\log n) = \mathcal{O}(n\log n).$$

The second step is doing an in-order traversal, in which case each node is visited exactly once, this means that the second step must be $\mathcal{O}(n)$.

Adding the two steps together gives:
$$\mathcal{O}(n\log n) + \mathcal{O}(n) = \mathcal{O}(n\log n).$$

Recall from the previous lecture, where we discussed sorting algorithms, that having a cost of $\mathcal{O}(n\log n)$ as an average cost of a sorting algorithm is actually really good, and theoretically it cannot get any better! Thus, simply by creating a binary search tree, we have found a sorting algorithm that scales well, much better than both bubble sort and selection sort.

#### What about the Worst case?

In our analysis, we used the average case of $\mathcal{O}(\log n)$ for insertions to find the cost of inserting the elements into the tree. But, what if the tree becomes unbalanced while inserting? In that case, the cost of an insertion becomes $\mathcal{O}(n)$ per insertion, much like the linked list. And so the total cost of inserting $n$ elements becomes $n\cdot\mathcal{O}(n) = \mathcal{O}(n^2)$. This also means that our whole sorting algorithm becomes $\mathcal{O}(n^2)$, the same as bubble sort and selection sort.

The trend that an unbalanced tree will perform poorly is starting to become a trend in this lecture. But for binary tree sorting especially, it turns out to become quite paradoxical. Recall from our example of the "perfectly" unbalanced tree, that we inserted the elements in sorted order into the tree, and this creates an unbalanced tree. In fact, our assumption of a balanced tree in the average case assumes the input data to be randomly ordered. This means that our binary search tree sort actually performs *best* on really well shuffled data, while if you hand it a sorted list, then it performs horribly.

## Self Balancing Trees

The main problem we keep meeting with the binary search tree data structure is that it can become unbalanced if the inserts into the tree come in nearly sorted or sorted order. However, this problem can be fixed by making the tree *self-balance*. Such self-balancing trees are trees that act like normal binary search trees, but they also self-monitor. If we would insert or delete a node that would cause things to become unbalanced, they will shift their nodes around to correct this. Much like the "resizing" of dynamic arrays, this operation occurs completely hidden from the users view, and is only carried out when needed.

The first self-balancing data structure was the AVL tree, which is just a specialized binary search tree. It is named after its inventor Adelson, Velskii and Landis. Other self-balancing trees have been invented since, for example splay trees, but AVL trees are still one of the simplest to implement and sees a lot of use. They published their data structre in the 1962 paper *An algorithm for the organization of information*. While self-balancing trees were a brand new concept in the sixties, today, almost any real-word use of binary search trees will be self-balancing, to ensure their performance.

The idea behind AVL trees is fairly simple. We just add an additional step after inserting and deleting nodes, that checks if the tree is still balanced after the operation. It it isn't, we take the time to balance it out there and then. Because the balancing happens after each insertion or deletion, the balancing itself is still fairly easy to carry out. The [Wikipedia article on AVL trees](https://en.wikipedia.org/wiki/AVL_tree) shows a gif of this process, we'll link it below:
<img src="fig/AVL_Tree_Example.gif">
<center><b>Source:</b> Created by <a href="https://commons.wikimedia.org/wiki/File:AVL_Tree_Example.gif">Bruno Schalch</a> under a <a href"https://creativecommons.org/licenses/by-sa/4.0/">CC BY-SA 4.0 license</a>.</center>

Implementing AVL trees is actually not that tricky, we could take the BST class we made, and just add a `self_balance` method to it, and we would be done. However, we have covered *a lot* on BSTs already, so we will skip this for now. If you are curious about AVL trees and want to implement the self-balancing aspect on your own, go for it! In that case, this [MIT 6.006 Lecture on AVL trees](https://ocw.mit.edu/courses/electrical-engineering-and-computer-science/6-006-introduction-to-algorithms-fall-2011/lecture-videos/lecture-6-avl-trees-avl-sort/) could be very helpful (the full lecture is about 50 minutes but also contains some review about BSTs).

### BST Sort with self-balancing trees

If we had implemented a self-balancing tree, we would guarantee the performance of $\mathcal{O}(\log n)$ insertions, and followingly guaranteed a sorting algorithm of $\mathcal{O}(n\log n)$, also in the worst case. However, as we need to insert all $n$ elements, and write them out again, the best case is also $\mathcal{O}(n\log n)$. And so for "sorting" already sorted lists, bubble sort beats our BST sort.

Looking back to the [Wikipedia Article on Sorting Algorithms](https://en.wikipedia.org/wiki/Sorting_algorithm#Comparison_of_algorithms). We see that the Binary Tree Sort entry indeed has a performance of $\mathcal{O}(n\log n)$ in the best, average and worst case. What it doesn't have good performance on however, is *memory*. Because, if we want to sort a list of numbers. Then doing BST sort on them, we have to first create a complete tree as well, this means we basically use twice the memory. If we are sorting extremely large lists, we do not want to do this, and we would much rather choose an algorithm that sorts in-place, so no extra memory is needed. Quicksort for example, does this.

If you have a list of numbers you want to sort, BST sort isn't necessarily the best way to go about it. But if you need to store data, and often want to access it in a sorted manner, then storing it in a BST is probably better than a list. As we learned last week, appending elements to a list is $\mathcal{O}(1)$, but if you want to ensure it is sorted, this will cost you! For a BST, you pay the slightly higher price of $\mathcal{O}(\log n)$ for the insertion, but you don't have to sort it, you get that for free! 









## Indexing Binary Search Trees

One final thing to discuss before leaving BSTs and data structures behind for this time: indexing. So far, we have not discussed indexing a binary search tree, or implemented the `operator[]` method. Now, you might think we haven't done this, because the BST doesn't store data in a *sequence*. And while it is true that the BST doesn't store things in a sequence in the sense that it doesn't remember its insertion order, it is still stores $n$ elements in a given order, namely: sorted order. It is therefore possible to index elements by their position in the sorted order.

To implement this, the most intuitive approach would probably be to simply do in-order traversal, like we for example do with printing, untill we have iterated through $i$ elements, in which case we can return the element at the correct index. This would work well, but would require traversing over $\mathcal{O}(i)$ steps, and so give a similar performance to indexing a linked list, i.e., $\mathcal{O}(n)$, a fairly bad scaling.

However, there is another approach we can take if we want to implement indexing. In our `BinarySearchTree` class, we have kept count of the number of elements stored in the complete tree, however, a subtree doesn't know how many elements it has. We can change this by moving the `count` variable into each node. Thus, `root.count` would be the number of elements in the entire tree, but any subnode would also know how many elements there were in its own subtree.

With this knowledge implemented at each node, and ensuring that we update the count of each node at each insertion and removal, an easy task with the recursive implementation, we could implement indexing by figuring if we had to follow the left or right branch at each node. For example, if the left branch contains a total of 10 nodes, and are trying to reach index 14, we would follow the right branch. This algorithm could thus only move down in the tree, unlike our traversal solution, and so the performance would be $\mathcal{O}(\log n)$. 

Thus, indexing a BST costs $\mathcal{O}(\log n)$, which is better than for a linked list, which was $\mathcal{O}(n)$, but worse than for dynamic arrays, where it is $\mathcal{O}(1)$. However, do remember that indexing means different things for the different structures. For the lists, it is getting the $i$'th element, based on insertion order. While for a BST, it means getting the $i$'th element in the sorted list, for example the $i$'th smallest element for a list of numbers.




## Wrapping up on Data Structures and Algorithms

To wrap up our discussion on data structures in IN1910, let us list the analysis we did on our different data structures

| Operation           | Array            | Dynamic Array      | Linked List      | Binary Search Tree$^{**}$ |
|---------------------|------------------|--------------------|------------------|--------------------|
| Indexing            | $\mathcal{O}(1)$ | $\mathcal{O}(1)$   | $\mathcal{O}(n)$ | $\mathcal{O}(\log n)$ |
| Insert at beginning | -                | $\mathcal{O}(n)$   | $\mathcal{O}(1)$ | $\mathcal{O}(\log n)$ |
| Insert at end       | -                | $\mathcal{O}(1)^*$ | $\mathcal{O}(1)$ | $\mathcal{O}(\log n)$ |
| Insert in middle    | -                | $\mathcal{O}(n)$   | $\mathcal{O}(n)^{***}$ | $\mathcal{O}(\log n)$ |
| Search              | $\mathcal{O}(n)$ | $\mathcal{O}(n)$   | $\mathcal{O}(n)$  | $\mathcal{O}(\log n)$ |

\*) Amortized/averaged cost \*\*) assuming balanced binary tree, also note that insertion is a single method on the BST, as the concept of "position" in the tree is different. \*\*\*) Insertion itself is $\mathcal{O}(1)$, but indexing into the list is $\mathcal{O}(n)$.